In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('/kaggle/input/goodreads-or-badreads/train.csv')


In [ ]:
data =  pd.read_csv('/kaggle/input/pre-data/pre_data.csv')

In [ ]:
data = data[['review_text','rating']]
data.shape

(630000, 2)

In [ ]:
data['review_text'] = data['review_text'].astype(str)

In [ ]:
test = pd.read_csv('/kaggle/input/goodreads-or-badreads/test.csv')

In [ ]:
train_df = data.iloc[:600000,:]
valid_df = data.iloc[600000:615000,:]
test_df = data.iloc[615000:630000,:]

In [ ]:
test_df = test[['review_text']]
test_df.head()

,review_text
0,I honestly wasn't expecting much from this boo...
1,I really enjoyed this book. It was well writte...
2,I am so thrilled to be finally reading this bo...
3,The premise of this book was right up my alley...
4,"So, I am a fan of genre-y novels, and as such ..."


In [ ]:
train_df.value_counts('rating')

rating
4    209227
5    176767
3    125891
2     48352
0     20647
1     19116
Name: count, dtype: int64

In [ ]:
from transformers import BertTokenizer
PRETRAINED_LM = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_LM, do_lower_case=True)
tokenizer

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
def encode(docs):
    '''
    This function takes list of texts and returns input_ids and attention_mask of texts
    '''
    encoded_dict = tokenizer.batch_encode_plus(docs, add_special_tokens=True, max_length=128, padding='max_length',
                            return_attention_mask=True, truncation=True, return_tensors='pt')
    input_ids = encoded_dict['input_ids']
    attention_masks = encoded_dict['attention_mask']
    return input_ids, attention_masks

In [ ]:
!pip install gdown
import gdown

In [ ]:
!gdown --id 1n9wMw_oEUsIrCG3hTNbbKjyrp1TVoYcS
!gdown --id 1VsSc5_PLxXI2o6sFoG5tUnaUgOPtXpqi
#https://drive.google.com/file/d/1lQ4MEHhUhtkH8ul7PW49IhcuYmjue-64/view?usp=sharing
!gdown --id 1lQ4MEHhUhtkH8ul7PW49IhcuYmjue-64
#https://drive.google.com/file/d/1XqX3H1uaTG9qnyB6MAavflJ9BpsF9eLp/view?usp=sharing
!gdown --id 1XqX3H1uaTG9qnyB6MAavflJ9BpsF9eLp

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1n9wMw_oEUsIrCG3hTNbbKjyrp1TVoYcS
From (redirected): https://drive.google.com/uc?id=1n9wMw_oEUsIrCG3hTNbbKjyrp1TVoYcS&confirm=t&uuid=73e4e163-1db2-440c-b3d5-1ff3c30bd565
To: /kaggle/working/valid_att_masks.pt
100%|█████████████████████████████████████████| 258M/258M [00:01<00:00, 157MB/s]
/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1VsSc5_PLxXI2o6sFoG5tUnaUgOPtXpqi
From (redirected): https://drive.google.com/uc?id=1VsSc5_PLxXI2o6sFoG5tUnaUgOPtXpqi&confirm=t&uuid=4fd7527a-1ffb-4d46-

In [ ]:
train_input_ids, train_att_masks = encode(train_df['review_text'].values.tolist())
valid_input_ids, valid_att_masks = encode(valid_df['review_text'].values.tolist())
test_input_ids, test_att_masks = encode(test_df['review_text'].values.tolist())

In [ ]:
torch.save(train_input_ids,'train_input_ids.pt')
torch.save(train_att_masks,'train_att_masks.pt')
torch.save(valid_input_ids,'valid_input_ids.pt')
torch.save(valid_att_masks,'valid_att_masks.pt')
torch.save(test_input_ids,'test_input_ids.pt')
torch.save(test_att_masks,'test_att_masks.pt')

In [ ]:
import torch
train_att_masks = torch.load('./train_att_masks.pt')
train_input_ids = torch.load('./train_input_ids.pt')
valid_att_masks = torch.load('./valid_att_masks.pt')
valid_input_ids = torch.load('./valid_input_ids.pt')

In [ ]:
train_att_masks.shap

torch.Size([504000, 256])

In [ ]:
test_input_ids, test_att_masks = encode(test_df['review_text'].values.tolist())

In [ ]:
temp = data.iloc[:270000,:]
temp = temp[['rating']]


(270000, 1)

In [ ]:
test_df['rating']=temp

/tmp/ipykernel_104/485954191.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['rating']=temp


In [ ]:
test_df.head()

,review_text,rating
0,I honestly wasn't expecting much from this boo...,3
1,I really enjoyed this book. It was well writte...,5
2,I am so thrilled to be finally reading this bo...,4
3,The premise of this book was right up my alley...,5
4,"So, I am a fan of genre-y novels, and as such ...",5


In [ ]:
import torch
train_y = torch.LongTensor(train_df['rating'].values.tolist())
valid_y = torch.LongTensor(valid_df['rating'].values.tolist())
test_y = torch.LongTensor(test_df['rating'].values.tolist())
train_y.size(),valid_y.size(),test_y.size()

(torch.Size([600000]), torch.Size([15000]), torch.Size([15000]))

In [ ]:
test_y = torch.LongTensor(test_df['rating'].values.tolist())
test_y.size()

torch.Size([270000])

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

BATCH_SIZE = 32
train_dataset = TensorDataset(train_input_ids, train_att_masks, train_y)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)

valid_dataset = TensorDataset(valid_input_ids, valid_att_masks, valid_y)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=BATCH_SIZE)

test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
test_dataset = TensorDataset(test_input_ids, test_att_masks, test_y)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [ ]:
from transformers import BertForSequenceClassification
N_labels = 6
model = BertForSequenceClassification.from_pretrained(PRETRAINED_LM,
                                                      num_labels=N_labels,
                                                      output_attentions=False,
                                                      output_hidden_states=False
                                                     )

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model = model.cuda()

In [ ]:
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

EPOCHS = 30
LEARNING_RATE = 2e-6

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer,
             num_warmup_steps=0,
            num_training_steps=len(train_dataloader)*EPOCHS )

In [ ]:
from torch.nn.utils import clip_grad_norm_
from tqdm.notebook import tqdm
import numpy as np
import math

train_loss_per_epoch = []
val_loss_per_epoch = []


for epoch_num in range(EPOCHS):
    print('Epoch: ', epoch_num + 1)
    '''
    Training
    '''
    model.train()
    train_loss = 0
    for step_num, batch_data in enumerate(tqdm(train_dataloader,desc='Training')):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        train_loss += loss.item()

        model.zero_grad()
        loss.backward()
        del loss

        clip_grad_norm_(parameters=model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    train_loss_per_epoch.append(train_loss / (step_num + 1))


    '''
    Validation
    '''
    model.eval()
    valid_loss = 0
    valid_pred = []
    with torch.no_grad():
        for step_num_e, batch_data in enumerate(tqdm(valid_dataloader,desc='Validation')):
            input_ids, att_mask, labels = [data.to(device) for data in batch_data]
            output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

            loss = output.loss
            valid_loss += loss.item()

            valid_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))

    val_loss_per_epoch.append(valid_loss / (step_num_e + 1))
    valid_pred = np.concatenate(valid_pred)

    '''
    Loss message
    '''
    print("{0}/{1} train loss: {2} ".format(step_num+1, math.ceil(len(train_df) / BATCH_SIZE), train_loss / (step_num + 1)))
    print("{0}/{1} val loss: {2} ".format(step_num_e+1, math.ceil(len(valid_df) / BATCH_SIZE), valid_loss / (step_num_e + 1)))

Epoch:  1


Training:   0%|          | 0/18750 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print('classifiation report')
print(classification_report(valid_pred, valid_df['rating'].to_numpy()))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


classifiation report
              precision    recall  f1-score   support

           0       0.41      0.54      0.47       291
           1       0.40      0.42      0.41       272
           2       0.35      0.49      0.41       598
           3       0.51      0.53      0.52      2026
           4       0.61      0.53      0.57      3881
           5       0.65      0.66      0.65      2932

    accuracy                           0.56     10000
   macro avg       0.49      0.53      0.50     10000
weighted avg       0.57      0.56      0.57     10000



In [ ]:
model.eval()
test_pred = []
test_loss= 0
with torch.no_grad():
    for step_num, batch_data in tqdm(enumerate(test_dataloader)):
        input_ids, att_mask, labels = [data.to(device) for data in batch_data]
        output = model(input_ids = input_ids, attention_mask=att_mask, labels= labels)

        loss = output.loss
        test_loss += loss.item()

        test_pred.append(np.argmax(output.logits.cpu().detach().numpy(),axis=-1))
test_pred = np.concatenate(test_pred)

0it [00:00, ?it/s]

In [ ]:
test_pred

NameError: name 'test_pred' is not defined

In [ ]:
test_pred.to_csv('Pred.csv',index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [ ]:
print('classifiation report')
print(classification_report(test_pred, test_df['rating'].to_numpy()))

classifiation report
              precision    recall  f1-score   support

           0       0.31      0.51      0.38       199
           1       0.26      0.49      0.34       164
           2       0.40      0.47      0.44       709
           3       0.55      0.52      0.53      2284
           4       0.55      0.54      0.54      3471
           5       0.69      0.65      0.67      3173

    accuracy                           0.56     10000
   macro avg       0.46      0.53      0.49     10000
weighted avg       0.58      0.56      0.57     10000

